In [1]:
import feedparser
podcast_feed_url = "https://access.acast.com/rss/d556eb54-6160-4c85-95f4-47d9f5216c49"
podcast_feed = feedparser.parse(podcast_feed_url)

In [2]:
print ("The number of podcast entries is ", len(podcast_feed.entries))

The number of podcast entries is  1599


In [3]:
import os
import requests


# Ensure the download directory exists
download_dir = "./content/audio"
os.makedirs(download_dir, exist_ok=True)


episode_url = podcast_feed.entries[0].enclosures[0].href
response = requests.get(episode_url, stream=True)

if response.status_code == 200:
    with open("./content/audio/podcast_episode.mp3", "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print("Download complete!")
else:
    print("Failed to download:", response.status_code)


Download complete!


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
%%time

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cuda:0


CPU times: user 6.99 s, sys: 1.07 s, total: 8.06 s
Wall time: 13.5 s


/home/teng/development/projects/public/podcast-audio-transcription/podcastenv/lib/python3.10/site-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


In [6]:
%%time

result = pipe("./content/audio/podcast_episode_6.mp3")
print(result["text"])

/home/teng/development/projects/public/podcast-audio-transcription/podcastenv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Back in 2016 as Colombia's violent paramilitary group, the FARC disbanded, Columbians hoped for a long peace. But in one region, another guerrilla group now wants supremacy and is terrorizing the public and hunting down former FARC members with impunity. And our obituaries editor pays tribute to Marianne faithful. For better and for worse, the British singer's life intersected the era of free love, the Rolling Stones, William S Burrows, and epic amounts of heroin, but hers is most of all a redemption story. First up though. Back in December South Korea's President Jun Seukjol sent troops streaming into the National Assembly, part of his attempt to impose martial law. As the chaos played out, E.J. Meung, leader of the opposition Democratic Party, streamed it. This is serious, vigorous democracy stuff. Mr. E s

In [7]:
# Check the transcription happened correctly by peeking into the first 1000 characters
podcast_transcript = result['text']
result['text'][:1000]

" The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Back in 2016 as Colombia's violent paramilitary group, the FARC disbanded, Columbians hoped for a long peace. But in one region, another guerrilla group now wants supremacy and is terrorizing the public and hunting down former FARC members with impunity. And our obituaries editor pays tribute to Marianne faithful. For better and for worse, the British singer's life intersected the era of free love, the Rolling Stones, William S Burrows, and epic amounts of heroin, but hers is most of all a redemption story. First up though. Back in December South Korea's President Jun Seukjol sent troops streaming into the National Assembly, part of his attempt to impose martial law. As the chaos played out, E.J. Meung, leader of the opposition Democratic Party, streamed it. This is serious, vigorous democracy stuff. Mr. E 

In [8]:
podcast_transcript

" The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Back in 2016 as Colombia's violent paramilitary group, the FARC disbanded, Columbians hoped for a long peace. But in one region, another guerrilla group now wants supremacy and is terrorizing the public and hunting down former FARC members with impunity. And our obituaries editor pays tribute to Marianne faithful. For better and for worse, the British singer's life intersected the era of free love, the Rolling Stones, William S Burrows, and epic amounts of heroin, but hers is most of all a redemption story. First up though. Back in December South Korea's President Jun Seukjol sent troops streaming into the National Assembly, part of his attempt to impose martial law. As the chaos played out, E.J. Meung, leader of the opposition Democratic Party, streamed it. This is serious, vigorous democracy stuff. Mr. E 

In [ ]:
podcast_transcript = """
  The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective 
  on the events shaping your world. Back in 2016 as Colombia's violent paramilitary group, the FARC disbanded, Columbians hoped for a long peace. 
  But in one region, another guerrilla group now wants supremacy and is terrorizing the public and hunting down former FARC members with impunity. 
  And our obituaries editor pays tribute to Marianne faithful. For better and for worse, the British singer's life intersected the era of free love, 
  the Rolling Stones, William S Burrows, and epic amounts of heroin, but hers is most of all a redemption story. First up though. Back in December 
  South Korea's President Jun Seukjol sent troops streaming into the National Assembly, part of his attempt to impose martial law. As the chaos 
  played out, E.J. Meung, leader of the opposition Democratic Party, streamed it. This is serious, vigorous democracy stuff. Mr. E scaled the walls 
  of Parliament to help put a stop to what was seen as a self-coo. It's been two months since that night and South Korea is still in limbo. 
  Mr. Eun and his Prime Minister were booted out. Mr. Eun was indicted last month for insurrection and his impeachment trial is still going. 
  So as the dust continues to settle, attention has turned to future leadership. Our East Asia Bureau Chief Noah Snyder went to meet the man who's 
  most likely to become the next president. Late last month my colleague Emma Hogan and I went to see E.J. Meyong at the same National Assembly 
  building which just several months earlier had been the site of this remarkable showdown on the night President Eun declared martial law. 
  E. J. Meung was confident, he was composed, and he was also quick to smile. He had the air of a practiced politician about him. Someone who can 
  spin answers off the cuff doesn't need to look at his notes, knows what he wants to talk about and what he wants to avoid. He made the point that 
  it's essential now to restore constitutional order. And he skirted questions about running for president again, which he may have to answer directly 
  very soon. If he did run and did win, it would have huge implications for South Korea's future direction across a host of different domains. 
  So, I mean for a start, how likely would he be to win? It's a good question and there's some important caveats to be made here. In the polling at 
  the moment, he is the front runner. That said, he faces some serious legal problems of his own. He's been charged with a spate of alleged crimes 
  linked to a development project during his time as mayor of his hometown south of Seoul and to some illegal fund transfers to North Korea. 
  In November of last year he was found guilty on election law violations for lying during the previous presidential campaign, which strikingly 
  is a crime in South Korea. and if that conviction is upheld on appeal he'd be barred for running for several years. Now he denies any wrongdoing 
  and says these charges are essentially politically motivated. His allies remain pretty confident that he'll be able to run in snap presidential 
  elections because the appeals process on any of these charges will take much longer than it will to get essentially to the polls. Regardless of 
  where the legal processes end up, the allegations alone have helped to make him a really divisive figure in South Korea, someone who inspires fierce, 
  passionate opinions from both supporters and opponents. And we saw this really manifest clearly ahead of parliamentary elections last year when 
  he survived an assassination attempt by a knife-wielding extremist. And so all of that is feeding into or perhaps just reflective of the very 
  polarized political environment that we spoke to you about last time. Exactly and in many ways that polarization is becoming ever more entrenched 
  in recent weeks the public opinion polls which had been strongly in favor of his party, the Democratic Party following the failed martial law 
  attempt, they've swung back. Some polls show them essentially neck and neck. There's a frustration that voters are now expressing towards the 
  Democratic Party. And we asked him about what he reads into these shifts in public opinion. His view was that it's a sign that voters saw his party 
  as an opposition force in the past but now essentially see it as the main political actor, effectively the leadership force, and that they need to 
  take more responsibility for bringing the turmoil to a swift and smooth end. So what does that pitch to voters look like? Well, E.J. Meung in recent 
  weeks has really been playing up the idea that he is a responsible choice. Now it's worth stepping back and sketching a little bit of a portrait of 
  this man because his life story is quite remarkable. He grew up in abject poverty, his parents cleaned public toilets, he couldn't go to secondary 
  school because he had to work in factories to help earn money for the family, And he ended up essentially educating himself and getting into 
  law school. He became a prominent human rights lawyer and labor activist before entering into politics. And he rose quite rapidly through the 
  political system in South Korea where he developed a reputation as someone who's extremely effective but at times slippery hard to pin down a bit 
  of an operator never quite sure what exactly he stands for. What he's saying to us is that what he stands for is pragmatism, realism, attack 
  towards the center, especially on economic issues where he was known as a somewhat of a progressive in the past. He talked a lot to us about the 
  importance of restoring growth, the crucial role of private markets and so on. And he also hinted at a potentially more pragmatic approach to 
  foreign policy than we've seen from Democratic Party administrations in the past. That's a bit of a tease there, Noah. What do you mean? What's the 
  interesting foreign policy angle here? Well, the Democratic Party is typically known to favor engagement with North Korea. It typically mistrust 
  Japan due to its abuses during the colonial period. And while supportive of the alliance with America, it seeks a greater balance between America 
  and other great powers in particular China. But in meetings with foreign diplomats and in our interview as well, he tried to dispel some of those 
  assumptions. He positioned South Korea pretty firmly as a member of the what he called liberal democratic camp diplomatic code for essentially 
  the Western side of the line in the American Chinese confrontation. There are forces that perhaps pull him in a slightly different direction than 
  his predecessors. That said, we shouldn't overstate this. A change in administration in South Korea would still mean a big change in South Korea's 
  foreign policy orientation. You hinted at that at the beginning that what happens here might really change the course for South Korea and for the 
  region. I mean, give me some detail. I think you can look at South Korea's foreign policy through three interlocking challenges. The first and most 
  fundamental is the one with North Korea. Now, as we talked about earlier, the left tends to favor more engagement and certainly Ijay-Mion would 
  shift the balance between deterrence and dialogue more in the direction of dialogue. But he was also clear that he seized the relationship with 
  North Korea at this stage as, quote, a hostile one. Strikingly, when talking about Japan he was also clear that at this stage it is not a hostile 
  relationship and he sounded to my ears more open than past leaders of his party to collaboration and cooperation with Japan. That said, he was clear 
  that he didn't want South Korea to be in a quote subordinate relationship and when it came to America and China this was the place where he was 
  the slipperiest he talked about needing to be practical and avoiding spoiling relations with China to the point where it hurts South Korea's own 
  national interest. I suspect that IJMion, if he does become South Korea's next president, we'll find that navigating this superpower competition 
  and this turbulent geopolitical environment around South Korea will make the defense of democracy at home feel like the easy part. Noah, thanks very 
  much for your time. Thank you for having me as always. The Last month violence erupted in the Colombian region of Catatumbo that's in the northeast 
  of the country on the border with Venezuela. Claire McHugh writes about Latin America for the economist and is based in Colombia. And things really 
  escalated on January the 16th when the oldest active guerrilla group called the National Liberation Army or the E LN as they're known over here, 
  they began attacking dissidents of the revolutionary armed forces of Colombia who are better known as the FARC, they demobilized under the 
  Peace Hill of 2016. And the nature of the attack is horrific. Residents report fighters going door to door, assassinating community leaders and 
  picking off former FARC members who signed that peace deal. The death toll continues to rise. The last 8th year has 7thas in the region of the 
  Kata Tumbo, Madhienta personas. At least 80 have been killed so far, for locals estimate the true number to be in the hundreds. This is one of 
  Colombia's worst security crises for decades. Locals from Katowboe, have escaped their homes in canoes, in cars, in motorcycles. It's a really 
  tragic situation. So tell me more about the crisis. What's going on here? More than 53,000 people have been displaced so far. It's the largest 
  forced movement of people in the country since records began in 1997. Almost half of them have fled to the regional capital of Kukutatah, which 
  is directly on the border with Venezuela. They're taking refuge in the football stadium there in the center of the city that's become the site 
  of the humanitarian response. And I spoke to some of the people about their experience of fleeing their homes. Many of them were terrified. 
  They're worried about being a target for these armed groups in future. A woman I spoke to told me that she locked herself in her house for four 
  days with her children while bodies piled up on the street outside. Another girl who was a teenager she fled with her 10-year-old brother and 
  her mother and had to pass through checkpoints with masked men. I spoke to these people at the stadium in Kukuta where they're waiting for food 
  and shelter and there's children there receiving packets of coloring pencils and in the center of the stadium there's a chef who's making more 
  than 15,000 meals a day for these people. He's cooking big fats of San 1,000 kilo of protein. In 3 po, I carne, yes, I may not see going to be 
  a hornada. But not everyone managed to get out of their homes. About 31,000 more people are still trapped in Katowo, that had to front of the 
  situation. I met a Red Cross coordinator who's part of the humanitarian response. And he was explaining to me that some of the children who left 
  have PTSD because they've seen their parents die. So it's a really tragic situation and this is not the first time that these people have faced 
  violence in their lives. So what is behind this particular flare-up in the violence though? As is always the case in Colombia, the heart of the 
  issue is cocaine. So Katatumbo is a hub of drug production. It's where about 17% of Colombia's coca leaf is grown. Coca is the base ingredient 
  for cocaine. So as a result of this, violence has long been present in the region and the state has not been present. These armed groups have 
  started fighting for control of drug production and trafficking out through Venezuela. They did have a truce that I think that broke down towards 
  the end of last year which is why the fighting has returned and above all it's really a blow for President Kosovo Petro whose flagship policy was 
  total peace and that involved simultaneously negotiating with armed groups across Colombia. He's culled off peace talks. I mean what is he doing 
  to quell the violence? So the peace talks with the earlend that suspended for now and then he declared a state of internal commotion which is the 
  first time that's happened in Colombia since 2008, and that's an emergency measure that allows him to pass security laws and impose taxes by decree 
  for 90 days temporarily by passing the normal legislative procedures through Congress. So the military also got sent in to assist with the 
  humanitarian response, but the complication there is that the government has a ceasefire with the FARC dissidents and it's difficult for the 
  Colombian military to know who is fighting because no one has insignia. So it's a really delicate situation for the military and it just highlights
  that the E. L. N. has near total control of that region of Colombia. How did it get to be in that kind of near total control? I mean the 
  E. L. N. operate across Colombia, but particularly in this region which is on the border with Venezuela, they benefit from being sheltered by 
  Venezuela where the regime of Nicholas Maduro profits from the illicit activities gold mining drug production that keep the ill and afloat and 
  continue to cause misery for Columbians. And actually the E. L. N. probably moved reinforcements from another region in Colombia through Venezuela. 
  So you've hinted that this is not the first time that the people have been subjected to this kind of violence, this flare-up between these armed 
  groups. What's the solution here? Is there a long-term way to keep the peace? I spoke to a group of community leaders from Katatumbo who I met in 
  Kukuta and they really emphasized to me that the territorial transformation is the only way really to resolve this conflict and that implies 
  reducing this region's dependence on cocaine and instead setting up alternative economies based on legal crops. But to do that, that needs to 
  have money from the state to implement these sorts of social programs, but also there needs to be enough money to have a clear security policy 
  designed. Now America's 90 days free on, freeze on foreign aid will also reduce the state's capacity to respond to this humanitarian crisis. 
  And in terms of the territorial transformation, the government of Gustavo Petro in 2022 agreed to build roads and schools and to create a kind 
  of agricultural superpower out of Catatumbo by encouraging farmers to switch from growing coca and instead growing high value crops such as cacao. 
  Unfortunately, no payments or roads or schools have materialized yet and those social leaders emphasized how deeply disappointed they were with 
  the government for that. Although you can't entirely blame the current government because a lot of this was foreseen by the peace deal of 2016, 
  most of which has not been fully implemented. And as long as that peace deal of 2016 is not fully implemented, then that cycle of violence in 
  Catatomb boat but across the whole of Colombia is likely to continue. Claire, thanks very much for your time. Thank you, Jason. Thanks for having 
  me on. When Marianne Faithful was asked what she remembered of the 60s, she had a whole list of things. Anne Row is the economist's obituaries 
  editor. Embroidered Moroccan cushions for one. Free love of course. Psychodelic drugs, knee-high boots, Ozzy Clark dresses, what she called 
  customised existentialism, the sort of place where you were wearing ruffled shirts and cloaks, and where if you wanted a good chat-up line 
  before jumping into bed with someone you might say have you read Lefler Dumas but what she remembered most of this extraordinary decade was 
  how beautiful everyone was. And she was most beautiful of all in many people's minds. Very few forgot her first appearance on top of the pops 
  in 1964, singing as tears go by, her first record. She looked like at a mere convent girl her long blonde hair was falling straight down her face, 
  her eyes were innocent, her voice was breathy and almost scared. That was when she a pedestal was as high as all that. She started by taking up 
  near constant smoking. But of course the aspect she concentrated on more than any other was sex, the great time of free love after the pill 
  had been invented. She tried three of the Rolling Stones. Brian Jones, she found a bit feeble and she didn't really fancy him. Keith Richards 
  gave her the greatest night of her life, but she decided the best bet was the lead singer, Mick Jagger, and so it was Mick who became her 
  boyfriend for the next four years. She had an image in that time of being an extremely sexy article. But she was getting into perilous waters 
  and the most dangerous of them of course were the drugs that were flowing at that time. She always insisted that she hadn't been on drugs before 
  she met the Rolling Stones. She dated her initiation to a summer party she went to in 1967, where the host had laid out six lines of cocaine on 
  the table. And he handed her a dollar bill. She didn't really know what to do with it, but when he told her she then snorted all six lines. 
  It was very weak and delicate cocaine, she said, apologetically afterwards, but it didn't take her long to go from cocaine to heroin. And she 
  was already quite strongly on heroin by the late 60s. In 1967 there came one of the most celebrated events of her career, which absolutely 
  delighted the press, who'd christened her now Mix's muse. They were told that there'd been a party going on at Keith Richards's country house. 
  The police went to raid it and when they found Marianne there she was wearing only a fur rug and a Mars bar which was rather intimately inserted 
  into her person. From then on the press had only to call her wanton woman in a fur rug or sometimes just Mars Bar. And she was utterly humiliated 
  and horrified by this, but it unfortunately stayed. But she began to feel after that that she really ought to disappear. So following the lead 
  of William Burras and what he'd written in the naked lunch, she decided she would go and be a junkie and live on the street. So that was what 
  she did for two years, 1970 and 1971. She more or less took nothing with her but her fashion dresses. And even though she would sit on a wall 
  in Soho wearing these fashion dresses, no one seemed to recognize her. She found that people were very kind to her the shopkeeper's roundabout 
  would wash her clothes for her and they would bring her cups of tea. She ran up the most enormous bill for drugs which she reckoned at 20,000 
  pounds. She was a bad junkie though in the sense that she couldn't even manage to inject her heroin herself. She kept missing the veins. And so 
  she became pock-marked. Her face was ravaged, she lost two teeth in a brawl. Her looks were completely gone, and her voice was gone when in 1979, 
  at the insistence of a record producer who rediscovered her, she recorded an album called Broken English. And you could hear on the title track 
  how her voice had changed. It's thrilling in a way to see how a human being can degrade herself and yet come through defiantly and defiance was 
  really the keynote to the rest of her life. She finally got clear of heroin in 1985. She had had a pretty terrible time in the end in the 60s, 
  but she'd also lived through a time when young people, these beautiful, scintillating folk believe that they were in the best time ever and that 
  there'd never been a better time to be a lie. And she was aware as much as anyone that they were all dancing on the edge of a volcano which 
  fairly soon was simply going to explode and blow them all away and she wasn't going to have regrets about a thing. Ann Row on Marianne Faithful 
  who's died aged 78. That's all for this episode of the Intelligence. We'll see you back here tomorrow. So, Oh, You're going to.
"""